In [1]:
# import
from arm import BernoulliArm
from greedy import Greedy

<BernoulliArm> mu: 0.8 hit/trial: 0/0
<Greedy> N=1000 Arms=[BernoulliArm(0.1), BernoulliArm(0.6), BernoulliArm(0.8), BernoulliArm(0.2)]


In [5]:
def mean_cumulative_reward(df_list):
    """累積報酬の平均を算出する
    
    :param df_list: DataFrameのリスト
    :return reward
    """
    return sum([df["reward"].sum() for df in df_list])

In [ ]:
mus = [0.1, 0.6, 0.8, 0.2]
arms = [BernoulliArm(mu) for mu in mus]
N=1000
alg = Greedy(arms,N)
print(alg)

In [2]:
df = alg.execute()

In [6]:
mean_cumulative_reward([df])

637